In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import datetime as dt
import warnings

# Part 1 - Long-only momentum strategy on sectors (Local analysis)

## I - Uniform Weights

In [ ]:
warnings.filterwarnings('ignore')

# Load data from Excel files
weights_df = pd.read_excel("Data SP.xlsx", sheet_name="Poids", index_col=0)
data_df = pd.read_excel("Data SP.xlsx", sheet_name="Données", index_col=0)

# Calculate sector index returns
returns_df = data_df[data_df.columns[::3]]
returns_df = returns_df.iloc[:, :-1]
returns_df.columns = data_df.columns[2::3]
returns_df = returns_df.iloc[1:].pct_change()
SP500 = returns_df.iloc[:, 0]
returns_df = returns_df.iloc[:, 1:]

# Calculate momentum indicators (3-month moving average)
momentum_df = returns_df.rolling(window=63).mean()  # 63 days represent approximately 3 months of trading data

# Select sectors to buy (those with the strongest momentum)
sectors_to_buy = momentum_df.iloc[-252].nlargest(3).index  # Buy the 3 sectors with the strongest momentum

# Build portfolio (allocation equal to selected sectors)
allocation = 1 / len(sectors_to_buy)
portfolio = pd.Series(index=sectors_to_buy, data=allocation)

# Calculate portfolio returns
portfolio_returns = (portfolio * returns_df[sectors_to_buy].iloc[-251:]).sum(axis=1)

# Calculate performance statistics
annual_return = (1 + portfolio_returns.mean()) ** 252 - 1  # 252 trading days in one year
annual_risk = portfolio_returns.std() * (252 ** 0.5)  # Annual standard deviation
maximum_drawdown = (portfolio_returns / portfolio_returns.cummax() - 1).min()

# Risk free rate
rf = (1+SP500.iloc[-251:].mean())**252 -1

# Print results
print("Annual return :", annual_return)
print("Annual risk :", annual_risk)
print("Maximum drawdown :", maximum_drawdown)
print("Sharpe ratio:", (annual_return- rf)/annual_risk)

Annual return : 0.5945605336074782
Annual risk : 0.2451259155417119
Maximum drawdown : -2.117020704541319
Sharpe ratio: 1.1490256540161292


In [ ]:
# Print S&P Analysis of 2023
print("Annual return :", rf)
print("Annual risk :", SP500.iloc[-251:].std() * (252 ** 0.5))
print("Maximum drawdown :", (SP500.iloc[-251:] / SP500.iloc[-251:].cummax() - 1).min())

Annual return : 0.3129045681858602
Annual risk : 0.11698892445906517
Maximum drawdown : -1.9374334156647066


## II - Sector weights

In [ ]:
import pandas as pd

# Load data from Excel files
weights_df = pd.read_excel("Data SP.xlsx", sheet_name="Poids", index_col=0)
data_df = pd.read_excel("Data SP.xlsx", sheet_name="Données", index_col=0)

# Calculate sector index returns
returns_df = data_df[data_df.columns[::3]]
returns_df = returns_df.iloc[:, :-1]
returns_df.columns = data_df.columns[2::3]
returns_df = returns_df.iloc[1:].pct_change()
SP500 = returns_df.iloc[:, 0]
returns_df = returns_df.iloc[:, 1:]

# Calculate momentum indicators (3-month moving average)
momentum_df = returns_df.rolling(window=63).mean()  # 63 days represent approximately 3 months of trading data

# Select sectors to buy (those with the strongest momentum)
sectors_to_buy = momentum_df.iloc[-252].nlargest(3).index  # Buy the 3 sectors with the strongest momentum

# Use sector weights to adjust allocation
weights_df.index = weights_df.index + ' Index'
allocation = weights_df.loc[sectors_to_buy, 'Poids'] / weights_df.loc[sectors_to_buy, 'Poids'].sum()

# Build the portfolio (allocation weighted by the selected sectors' weights)
portfolio_returns = (allocation * returns_df[sectors_to_buy].iloc[-251:]).sum(axis=1)

# Calculate performance statistics
annual_return = (1 + portfolio_returns.mean()) ** 252 - 1  # 252 trading days in one year
annual_risk = portfolio_returns.std() * (252 ** 0.5)  # Annual standard deviation
maximum_drawdown = (portfolio_returns / portfolio_returns.cummax() - 1).min()

# Display the results
print("Annual return:", annual_return)
print("Annual risk:", annual_risk)
print("Maximum drawdown:", maximum_drawdown)
print("Sharpe ratio:", (annual_return - rf) / annual_risk)  # Assuming 'rf' is the risk-free rate

Annual return: 0.6435178346313046
Annual risk: 0.23184895537708694
Maximum drawdown: -2.00728338091191
Sharpe ratio: 1.425985577151831


# Part 2 - Long/short momentum strategy (Local analysis)

## I - Uniform Weights

In [ ]:
# Select sectors to buy (the top 3 with the highest momentum) and to short sell (the bottom 3 with the lowest momentum)
sectors_to_buy = momentum_df.iloc[-252].nlargest(3).index
sectors_to_short_sell = momentum_df.iloc[-252].nsmallest(3).index

# Calculate weights for each sector (equal allocation to selected sectors)
allocation_to_buy = 1 / len(sectors_to_buy)
allocation_to_short_sell = -1 / len(sectors_to_short_sell)

# Build the portfolio (equal allocation to selected sectors)
portfolio = pd.Series(index=data_df.columns[2::3], data=0)
portfolio[sectors_to_buy] = allocation_to_buy
portfolio[sectors_to_short_sell] = allocation_to_short_sell

# Calculate portfolio returns
portfolio_returns = (portfolio * returns_df.iloc[-251:]).sum(axis=1)

# Calculate performance statistics
annual_return = (1 + portfolio_returns.mean()) ** 252 - 1  # 252 trading days in one year
annual_risk = portfolio_returns.std() * (252 ** 0.5)  # Annual standard deviation
maximum_drawdown = (portfolio_returns / portfolio_returns.cummax() - 1).min()

# Display the results
print("Annual return:", annual_return)
print("Annual risk:", annual_risk)
print("Maximum drawdown:", maximum_drawdown)
print("Sharpe ratio:", (annual_return - rf) / annual_risk)  # Assuming 'rf' is the risk-free rate

Annual return: 0.2618513759282659
Annual risk: 0.24793472832348545
Maximum drawdown: -2.235614423139661
Sharpe ratio: -0.20591384112589572


## II - Sector weights

In [ ]:
# Select sectors to buy (the top 3 with the highest momentum) and to short sell (the bottom 3 with the lowest momentum)
sectors_to_buy = momentum_df.iloc[-252].nlargest(3).index
sectors_to_short_sell = momentum_df.iloc[-252].nsmallest(3).index

# Use sector weights to adjust the allocation
allocation_to_buy = weights_df.loc[sectors_to_buy, 'Poids'] / weights_df.loc[sectors_to_buy, 'Poids'].sum()
allocation_to_short_sell = weights_df.loc[sectors_to_short_sell, 'Poids'] / weights_df.loc[sectors_to_short_sell, 'Poids'].sum()

# Build the portfolio (allocation weighted by the weights of the selected sectors)
portfolio_returns = (allocation_to_buy * returns_df[sectors_to_buy].iloc[-251:]).sum(axis=1) + (allocation_to_short_sell * returns_df[sectors_to_short_sell].iloc[-251:]).sum(axis=1)

# Calculate performance statistics
annual_return = portfolio_returns.mean() * 252  # 252 trading days in one year
annual_risk = portfolio_returns.std() * (252 ** 0.5)  # Annual standard deviation
maximum_drawdown = (portfolio_returns / portfolio_returns.cummax() - 1).min()

# Display the results
print("Annual return:", annual_return)
print("Annual risk:", annual_risk)
print("Maximum drawdown:", maximum_drawdown)
print("Sharpe ratio:", (annual_return - returns_df.mean().mean() * 252) / annual_risk)

Annual return: 0.7386827152309159
Annual risk: 0.32987237410280174
Maximum drawdown: -1.8397057745652616
Sharpe ratio: 1.9385655795088756


# Part 3: Returns of relative strength portfolios (Long/Short strategy)

Our aim is to implement a Long/Short strategy using S&P 500 historical data.

## I - Overview of the dataset

In [ ]:
FILE_NAME = "sp_data.csv"

Our dataset contains the history since 1989 of the s&p 500 and all the level 2 sector indices (24 in total). Since the indices S5REDPIG and S5REAL started later (2001 and 2023) we only take them into account after these dates.

In [ ]:
class Strat:
    """General class to load and process the dataset"""
    def __init__(self, J: int, K: int):
        self.J = J
        self.K = K

    def load(self, filename: str):
        """Load the given dataframe and process it"""
        df = pd.read_csv(filename, sep=";")
        #df = df.dropna(axis=1)
        df["date"] = pd.to_datetime(df["date"], format=f"%d/%m/%Y")
        df = df.set_index("date")
        for col in df.columns:
            df[col] = df[col].str.replace(',', '.').astype(float)

        self.portfolio = pd.DataFrame(index = df.columns)
        self.df = df

        self.num_asset = len(df.columns)
        self.decile = self.num_asset//10

    def compute_returns(self):
        """Compute daily and monthly returns"""
        self.daily_returns = self.df.pct_change(axis=0)[1:]
        num_days = self.daily_returns.resample('M', closed="right").count()
        self.monthly_returns = self.daily_returns.add(1).resample('M', closed="right").prod()**(20/num_days)-1
        self.df_rolling_av = self.daily_returns.rolling(window=20*self.J).mean()

    def update_J_K(self, J, K):
        self.J = J
        self.K = K
        self.df_rolling_av = self.daily_returns.rolling(window=20*self.J).mean()

We plot below some data:

In [ ]:
J = 3 # Number of months to look at preceding the portfolio formation date
K = 3 # Holding period in month

In [ ]:
# Loading and processing the file
strat1 = Strat(J, K)
strat1.load(FILE_NAME)
strat1.compute_returns()
strat1.df

,SPX,S5SFTW,S5PHRM,S5CPGS,S5ENRSX,S5FDBT,S5TECH,S5RETL,S5BANKX,S5HCES,...,S5FDSR,S5HOUS,S5SSEQX,S5TRAN,S5HOTR,S5CODU,S5AUCO,S5COMS,S5REDPIG,S5REAL
date,,,,,,,,,,,,,,,,,,,,,
1992-01-01,417.09,57.74,122.24,78.82,92.50,104.49,71.44,103.97,81.25,125.71,...,85.69,78.57,34.51,97.53,61.84,94.59,57.56,106.52,NaN,NaN
1992-01-02,417.26,58.43,121.78,78.56,92.93,104.35,72.38,105.19,81.72,123.96,...,86.00,77.77,35.14,95.88,63.00,94.33,59.67,106.60,NaN,NaN
1992-01-03,419.34,58.23,122.19,78.72,93.08,105.40,72.79,104.61,82.17,124.75,...,86.73,77.25,35.12,96.61,63.59,94.35,61.79,109.57,NaN,NaN
1992-01-06,417.96,57.85,121.71,78.84,91.89,104.35,73.47,103.55,82.62,126.61,...,85.48,76.76,35.37,96.64,64.98,93.97,62.67,109.13,NaN,NaN
1992-01-07,417.40,58.42,122.15,78.91,90.01,103.62,75.13,102.98,82.37,125.03,...,84.09,76.73,36.55,97.43,66.55,94.30,61.95,109.49,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,5088.80,4565.25,1401.59,1067.10,652.35,792.53,3424.60,4345.11,365.88,2013.06,...,771.65,860.88,4161.81,1071.16,1678.70,428.80,110.42,621.49,112.67,242.29
2024-02-26,5069.53,4560.86,1397.05,1067.78,654.45,786.22,3404.09,4339.73,363.70,1997.85,...,777.23,855.28,4191.43,1064.84,1668.36,427.40,113.77,620.12,111.99,239.43
2024-02-27,5078.18,4559.52,1395.34,1069.36,651.63,784.21,3425.56,4348.58,366.97,1988.26,...,780.85,853.55,4179.84,1066.15,1669.09,428.28,114.12,619.89,113.17,239.36


In [ ]:
fig = px.line(strat1.df["SPX"], x = strat1.df.index, y = "SPX", title="S&P 500 since 1989")
fig.show()

In [ ]:
fig = px.line(strat1.daily_returns["SPX"], x = strat1.daily_returns.index, y = "SPX", title="S&P 500 daily returns since 1989")
fig.show()

## II - Looking at the J preceding months to invest in the next K months

Our aim is to implement a strategy that looks at the J preceding months (20 days per month) to invest immediately or with a week-lag in the next K months (Long best assets - Short worst assets in terms of historical return over the J months). And to invest each month in this way. We first set J = 3 months and K = 3 months, then we will try to change it to see how it impacts the returns.

### a) Investing immediately

In [ ]:
class Strat_long_short(Strat):
    """class used to simulate a long-short strategy over K months
      based on top decile/worst decile portfolios over the last J months"""

    def __init__(self, J: int, K: int):
        super().__init__(J, K)

    def best_worst_port(self) -> dict:
        """Return the top decile portfolio and the worst decile one using rolling average over the last J months"""
        return_j =  self.df_rolling_av
        # We only use S5REAL and S5REDPIG after they exist
        if self.df.index.get_loc(self.starting_day)+20*J > self.df.index.get_loc(dt.datetime(2001, 10, 9)):
            return_j.drop("S5REAL", axis = 1)
        if self.df.index.get_loc(self.starting_day)+20*J > self.df.index.get_loc(dt.datetime(2023, 3, 17)):
            return_j.drop("S5REDPIG", axis = 1)

        best = list(return_j.loc[self.starting_day].nlargest(self.decile).index)
        worst = list(return_j.loc[self.starting_day].nsmallest(self.decile).index)

        return {'best': best, 'worst': worst}

    def strat_j(self, long_only: bool = False) -> pd.DataFrame:
        """Return the optimal portfolio at the given date looking at the return of the J preceding months"""
        num_assets = len(self.df.columns)
        portfolio = pd.DataFrame([0.0 for _ in range(num_assets)],
                                 index = self.df.columns,
                                 columns=[self.starting_day])
        best_worst_assets = self.best_worst_port()
        num_selected_asset = 2*self.decile
        if long_only:
            num_selected_asset = self.decile
        for asset in best_worst_assets["best"]:
            portfolio.loc[asset] = 1/num_selected_asset
        for asset in best_worst_assets["worst"]:
            if not long_only:
                portfolio.loc[asset] = -1/num_selected_asset
        return portfolio

    def returns_holding_k(self, starting_day: dt.datetime, add_week: bool = False, long_only: bool = False) -> tuple:
        """Return (annualized) when starting investing at the given date,
        and holding K months. Give also the return of the S&P over the same period"""
        self.starting_day = starting_day
        portfolio = self.strat_j(long_only)
        holding_period = self.K*20
        num_date = self.df.index.get_loc(self.starting_day)+5*add_week
        final_returns = pd.DataFrame((self.df.iloc[num_date+holding_period-1]-self.df.iloc[num_date])
                                     /self.df.iloc[num_date],
                                     columns = [self.starting_day])
        index_return = final_returns.iloc[0,]
        return ((final_returns*portfolio+1)**(12/self.K)-1).sum(), (index_return+1)**(12/self.K)-1

    def returns_holding_k_total(self, num_starting_day: int, period_inter_investment: int = 20, add_week: bool = False, long_only: bool = False) -> pd.DataFrame:
        """"Give the return of the strategy: each month we invest for K months looking back at the J preceding months.
            Give also the return of the S&P."""
        num_days = len(self.df)
        starting_day = self.df.index[num_starting_day]
        returns = self.returns_holding_k(starting_day, add_week, long_only)[0]
        returns_sp = self.returns_holding_k(starting_day, add_week, long_only)[1]
        for m in range(num_starting_day,num_days-self.K*20+1-5*add_week*period_inter_investment, period_inter_investment+5*add_week):
            starting_day = self.df.index[m]
            ret_hold_k = self.returns_holding_k(starting_day, add_week, long_only)
            returns = pd.concat([returns, ret_hold_k[0]])
            returns_sp = pd.concat([returns_sp, ret_hold_k[1]])
        returns.name = "opt_portfolio"
        returns = pd.concat([returns, returns_sp], axis = 1)
        return returns

In [ ]:
def compute_statistics(returns: pd.Series, returns_sp: pd.Series) -> tuple:
    """Return a couple containing two lists: names of some statistics related to the strategy, and their value"""
    list_stat_names = ["Average return of the strategy",
                "Average return of the s&p",
                "Risk strategy",
                "Risk S&P",
                "Strategy Sharpe ratio (regarding S&P)",
                "Maximum drawdown strategy",
                "Maximum drawdown S&P"]

    av_ret_strat = returns.mean()
    av_ret_sp = returns_sp.mean()
    risk_strat = returns.std()
    risk_sp = returns_sp.std()
    sharpe_ratio = (av_ret_strat-av_ret_sp)/((returns- returns_sp).std())

    cumulative_returns_strat = (1 + returns).cumprod()
    running_max_strat = cumulative_returns_strat.cummax()
    max_drawdown_strat = ((cumulative_returns_strat - running_max_strat) / running_max_strat).min()

    cumulative_returns_sp = (1 + returns_sp).cumprod()
    running_max_sp = cumulative_returns_sp.cummax()
    max_drawdown_sp = ((cumulative_returns_sp - running_max_sp) / running_max_sp).min()

    list_stats = [av_ret_strat, av_ret_sp, risk_strat, risk_sp, sharpe_ratio, max_drawdown_strat, max_drawdown_sp]
    for i, x in enumerate(list_stats):
        list_stats[i] = f"{x:.4f}"

    return (list_stat_names, list_stats)

In [ ]:
def heatmap(results: pd.DataFrame, title: str, save: bool = False) -> None:
    """Visualisation function of a dataframe into a heatmap"""
    fig = px.imshow(results,
                    labels=dict(x="J", y="K"),
                    x=[str(col) for col in results.columns],
                    y=[str(index) for index in results.index],
                    text_auto=".2%",
                    color_continuous_scale='sunset')

    # Customize axis titles
    fig.update_xaxes(title="J")
    fig.update_yaxes(title="K")

    # Update layout for more clarity
    fig.update_layout(
    title={
        'text': title,
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        },
        xaxis=dict(side="top", title_standoff=10),
        margin=dict(t=100)
    )

    # Show the plot
    fig.show()
    if save:
        fig.write_image("image.png", scale=6)

#### i. Results with J = 3 months and K = 3 months

For each month we compute the return of each asset over the last J months ($20 \times J$ days) to get the optimal portfolios (long best/short worst), then we compute the return over the next K months ($20 \times K$ days) when holding this portfolio.

In [ ]:
# Parameters
J, K = 3, 3
num_starting_day = J*20 # Number of the starting day
period_inter_investment = 20 # We wait one month before investing again

In [ ]:
# Implementing the strategy, and computing the annualized return for each day
strat0 = Strat_long_short(J, K)
strat0.load(FILE_NAME)
strat0.compute_returns()
tot_returns = strat0.returns_holding_k_total(num_starting_day, period_inter_investment)
returns = tot_returns["opt_portfolio"]
returns_sp = tot_returns["SPX"]
statistics = compute_statistics(returns, returns_sp)

Plotting the returns:

In [ ]:
returns.name = "Annualized returns when holding K months"
returns = pd.DataFrame(returns)

fig = px.line(returns, x = returns.index, y = returns.columns[0], title = "Return of the long/short strategy with J=3 and K=3")
fig.show()

It shows that the return of the strategy varies strongly depending on the month on which we start the strategy. Additionnaly, during crisis periods, it can be seen that there is a strong variability of the return of the strategy.

In [ ]:
returns_sp.name = "Annualized returns of the S&P 500"
returns_sp = pd.DataFrame(returns_sp)

fig = px.line(returns_sp, x = returns_sp.index, y = returns_sp.columns[0], title = "Return of the S&P 500 over the same period")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Table(header=dict(values=['Statistic', 'Value']),
                 cells=dict(values=[statistics[0],statistics[1]]))])
fig.show()

With this strategy, we obtain an average return of $2.48\%$ with a risk of $22\%$. This is much lower than the s&p 500 hence a negative sharpe ratio.

#### ii. Comparing results when changing K and J

We now change the value of K and J to see if we can obtain better results. The results are summarized in the following table:

In [ ]:
number_months = [3, 6, 9, 12]
results = pd.DataFrame(index = number_months, columns = number_months)
sharpe_ratios = pd.DataFrame(index = number_months, columns = number_months)
results_lagged = pd.DataFrame(index = number_months, columns = number_months)
sharpe_ratios_lagged = pd.DataFrame(index = number_months, columns = number_months)

In [ ]:
for K in number_months:
    for J in number_months:
        strat0.update_J_K(J, K)
        tot_returns = strat0.returns_holding_k_total(num_starting_day, period_inter_investment)
        returns = tot_returns["opt_portfolio"]
        returns_sp = tot_returns["SPX"]
        results.loc[K, J] = returns.mean()
        sharpe_ratios.loc[K, J] = (results.loc[K, J] - returns_sp.mean())/(returns-returns_sp).std()

In [ ]:
heatmap(results, "Average Return depending on J and K")

In [ ]:
heatmap(sharpe_ratios, "Sharpe Ratio depending on J and K")

It can be seen that the optimum is reached for : K = 3 months and J = 6 months: meaning looking at the returns of the last 6 months to long the best, short the worst and hold the position for 3 months. In this case we obtain an average return of $5.42\%$, which is still lower than the S&P return, hence a sharpe ratio of $-0.16$.

### b) Investing with a week lag

We now implement the same strategy, but after determining the optimal portfolio, we wait one week before buying/selling the assets and then we hold it for K months. We also start with J = 3 months and K = 3 months:

In [ ]:
# Parameters
J, K = 3, 3
num_starting_day = J*20 # Number of the starting day
period_inter_investment = 20 # We wait one month before investing again
add_week = True

In [ ]:
# Implementing the strategy, and computing the annualized return for each day
strat_week_lag = Strat_long_short(J, K)
strat_week_lag.load(FILE_NAME)
strat_week_lag.compute_returns()
tot_returns_lagged = strat_week_lag.returns_holding_k_total(num_starting_day, period_inter_investment, add_week)
returns_lagged = tot_returns_lagged["opt_portfolio"]
returns_lagged_sp = tot_returns_lagged["SPX"]
statistics_lagged = compute_statistics(returns_lagged, returns_lagged_sp)

#### i. Results with J = 3 months and K = 3 months

In [ ]:
returns_lagged.name = "K-month returns with a week lag"
returns_lagged = pd.DataFrame(returns_lagged)

fig = px.line(returns_lagged, x = returns_lagged.index, y = returns_lagged.columns[0], title = "Return of the long/short strategy with a waiting week (J = 3 months, K = 3 months)")
fig.show()

In [ ]:
returns_lagged_sp.name = "Annualized returns of the S&P 500"
returns_lagged_sp = pd.DataFrame(returns_lagged_sp)

fig = px.line(returns_lagged_sp, x = returns_lagged_sp.index, y = returns_lagged_sp.columns[0], title = "Return of the S&P 500")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Table(header=dict(values=['Statistic', 'Value']),
                 cells=dict(values=[statistics_lagged[0],statistics_lagged[1]]))])
fig.show()

With this strategy we obtain a slightly lower average return of $2.30\%$, and still a negative sharpe ratio since the return is the lower than the S&P return.

#### ii. Comparing average return when changing K and J

We now change the value of K and J to see if we can obtain better results. The results are summarized in the following table:

In [ ]:
for K in number_months:
    for J in number_months:
        strat_week_lag.update_J_K(J, K)
        tot_returns_lagged = strat_week_lag.returns_holding_k_total(num_starting_day, period_inter_investment, add_week)
        returns_lagged = tot_returns_lagged["opt_portfolio"]
        returns_sp_lagged = tot_returns_lagged["SPX"]
        results_lagged.loc[K, J] = returns_lagged.mean()
        sharpe_ratios_lagged.loc[K, J] = (results_lagged.loc[K, J] - returns_sp_lagged.mean())/(returns_lagged-returns_sp_lagged).std()

In [ ]:
heatmap(results_lagged, "Average Return depending on J and K (Week lag strategy)", save = True)

In [ ]:
heatmap(sharpe_ratios_lagged, "Sharpe Ratio depending on J and K (Week lag strategy)", save = True)

In this case, the best strategy in average is also obtained for J = 6 months and K = 3 months. We then obtain and average return of $4.88\%$, for a sharpe ratio of $-0.17$.

# Part 4: Long only strategy

In this section, we adopt a similar strategy as in the previous part, but instead of long/short best/worst assets at each step, we just long the top decile assets every month. Also we will set: J = 6 months and K = 3 months (looking back at the 6 previous months to invest in the next 3 months)

In [ ]:
# Parameters
J, K = 6, 3
num_starting_day = J*20 # Number of the starting day
period_inter_investment = 20 # We wait one month before investing again
long_only = True

In [ ]:
# Implementing the strategy, and computing the annualized return for each day
strat_long = Strat_long_short(J, K)
strat_long.load(FILE_NAME)
strat_long.compute_returns()
tot_returns = strat_long.returns_holding_k_total(num_starting_day, period_inter_investment, long_only)
returns = tot_returns["opt_portfolio"]
returns_sp = tot_returns["SPX"]
statistics = compute_statistics(returns, returns_sp)

In [ ]:
returns.name = "Annualized returns when holding K months"
returns = pd.DataFrame(returns)

fig = px.line(returns, x = returns.index, y = returns.columns[0], title = "Return of the long/short strategy")
fig.show()

In [ ]:
returns_sp.name = "Annualized returns of the S&P 500"
returns_sp = pd.DataFrame(returns_sp)

fig = px.line(returns_sp, x = returns_sp.index, y = returns_sp.columns[0], title = "Return of the S&P 500 over the same period")
fig.show()

In [ ]:
fig = go.Figure(data=[go.Table(header=dict(values=['Statistic', 'Value']),
                 cells=dict(values=[statistics[0],statistics[1]]))])
fig.show()

We the long only strategy, we obtain a better average return of $5.5\%$ compared to the long/short strategy, and thus still a negative sharpe ratio ($-0.14$).  